# **Welcome to Crocoddyl**

Crocoddyl is an **optimal control library** designed for **robot control in contact-rich environments**. It leverages an efficient **Differential Dynamic Programming (DDP)** solver to compute **optimal trajectories** and **feedback gains** for complex robotic systems.  

To achieve high-performance control, Crocoddyl integrates **Pinocchio**, a state-of-the-art rigid body dynamics library, enabling fast computation of **robot dynamics and analytical derivatives**.  

### Multi-Contact Optimal Control Problem (MCOP)

Crocoddyl is specifically designed to handle **multi-contact optimal control problems (MCOPs)**, which take the following form:


\begin{aligned}
\mathbf{X}^*, \mathbf{U}^* =
\begin{Bmatrix} \mathbf{x}^*_0, \dots, \mathbf{x}^*_N \\
                \mathbf{u}^*_0, \dots, \mathbf{u}^*_N
\end{Bmatrix} &=
\arg\min_{\mathbf{X},\mathbf{U}} \sum_{k=1}^{N} \int_{t_k}^{t_k+\Delta t} \ell(\mathbf{x}, \mathbf{u})dt \\

&\text{subject to} \\

&\mathbf{\dot{x}} = \mathbf{f}(\mathbf{x}, \mathbf{u}),\\

&\mathbf{h}(\mathbf{x}, \mathbf{u}) = \mathbf{0}, \\

&\mathbf{\underline{g}} \leq \mathbf{g}(\mathbf{x}, \mathbf{u}) \leq \mathbf{\overline{g}}, \\

&\mathbf{x} \in \mathcal{X}, \quad \mathbf{u} \in \mathcal{U}.
\end{aligned}

where:  
- The **state** is given by $\mathbf{x} = (\mathbf{q}, \mathbf{v})$, which lies on a **Lie manifold** (e.g., $\mathbf{q} \in SE(3) \times \mathbb{R}^{n_j}$), where $n_j$ is the number of degrees of freedom of the robot.  
- The system exhibits **underactuated dynamics**, meaning the control input takes the form $\mathbf{u} = (\mathbf{0}, \boldsymbol{\tau})$.  
- $\mathcal{X}$ and $\mathcal{U}$ define the **state and control admissible sets**.  
- $\mathcal{K}$ represents the **contact constraints**.  

## Getting Started

To fully understand Crocoddyl’s **design and capabilities**, let's begin by exploring its fundamental concepts and framework. 🚀


### I. Action Models in Crocoddyl

In Crocoddyl, an **action model** encapsulates both **dynamics** and **cost models**. Each **node** in an optimal control problem is described using an action model. To formulate an optimal control problem, we must define:
- The **dynamics function**, which models system evolution.
- The **cost function**, which quantifies control objectives.
- Their **derivatives**, which enable efficient optimization.

To understand the mathematical foundation of an action model, we consider a **locally linearized** version of our optimal control problem:

$$\mathbf{X}^*(\mathbf{x}_0),\mathbf{U}^*(\mathbf{x}_0)
=
\arg\max_{\mathbf{X},\mathbf{U}} = cost_T(\delta\mathbf{x}_N) + \sum_{k=1}^N cost_t(\delta\mathbf{x}_k, \delta\mathbf{u}_k)$$
subject to
$$dynamics(\delta\mathbf{x}_{k+1},\delta\mathbf{x}_k,\delta\mathbf{u}_k)=\mathbf{0},$$

where
$$cost_T(\delta\mathbf{x}) = \frac{1}{2}
\begin{bmatrix} 
  1 \\ \delta\mathbf{x}
\end{bmatrix}^\top
\begin{bmatrix}
0 & \mathbf{\ell_x}^\top \\
\mathbf{\ell_x} & \mathbf{\ell_{xx}}
\end{bmatrix}
\begin{bmatrix}
  1 \\ \delta\mathbf{x}
\end{bmatrix}
$$

$$cost_t(\delta\mathbf{x},\delta\mathbf{u}) = \frac{1}{2}
\begin{bmatrix} 
  1 \\ \delta\mathbf{x} \\ \delta\mathbf{u}
\end{bmatrix}^\top
\begin{bmatrix}
0 & \mathbf{\ell_x}^\top & \mathbf{l_u}^\top\\
\mathbf{\ell_x} & \mathbf{\ell_{xx}} & \mathbf{\ell_{ux}}^\top\\
\mathbf{\ell_u} & \mathbf{\ell_{ux}} & \mathbf{\ell_{uu}}
\end{bmatrix}
\begin{bmatrix}
  1 \\ \delta\mathbf{x} \\ \delta\mathbf{u}
\end{bmatrix}
$$

$$
dynamics(\delta\mathbf{x}_{k+1},\delta\mathbf{x}_k,\delta\mathbf{u}_k) = \delta\mathbf{x}_{k+1} - (\mathbf{f_x}\delta\mathbf{x}_k + \mathbf{f_u}\delta\mathbf{u}_k)
$$

where an action model defines a time interval of this problem:
 - $actions = dynamics + cost$

### Key Properties of Action Models
- **Encapsulates system dynamics and cost functions** for an optimal control problem node.
- **Operates in discrete-time** (unlike continuous formulations).
- **Provides analytical derivatives** for efficient optimization, leveraging **Pinocchio** for computational efficiency.
- **Supports numerical/automati differentiation (NumDiff/AutoDiff) and code generation (CodeGen)** for debugging, prototyping, and deploying.

### Main Computation in an Action Model

Optimal control solvers often need to compute a search direction and try it along a step length. These computations are performed by Crocoddyl's `calc` and `calcDiff` functions. Concretely, these functions update the information of:
 - **`calc`**: update the next state, cost value, and constraint infeasibilities.
 $$\delta\mathbf{\dot{x}}_{k+1} = \mathbf{f}(\delta\mathbf{x}_k,\mathbf{u}_k)$$
 - **`calcDiff`**: update the derivatives of the dynamics, cost, and constraints (quadratic approximation)
 $$\mathbf{f_x}, \mathbf{f_u} \hspace{1em} (dynamics)$$
 $$\mathbf{\ell_x}, \mathbf{\ell_u}, \mathbf{\ell_{xx}}, \mathbf{\ell_{ux}}, \mathbf{\ell_{uu}} \hspace{1em} (cost)$$
 
To optimize the performance of the solver, Crocoddyl stores all relevant information inside **data** objects. This approach avoids dynamic memory reallocation during the calculations, which helps to speed up the process, especially when the solver is called iteratively in the optimization process. Creating each data object is done via `createData`.

With this understanding, we can now explore **how to implement and use action models in Crocoddyl** in `02_optimizing_a_cartpole.ipynb`.

---

### II. States in Crocoddyl


In general, the system's state lies in a manifold $M$, where the rate of change of the state lies in its tangent space $T_\mathbf{x}M$. To perform operations within our solvers, we define the following operators:

- **Integration**: The operation to compute the next state given the current state and the change in state:
  
  \begin{equation}
  \mathbf{x}_{k+1} = \text{integrate}(\mathbf{x}_k, \delta \mathbf{x}_k) = \mathbf{x}_k \oplus \delta \mathbf{x}_k
  \end{equation}
  
- **Difference**: The operation to compute the change in state given two consecutive states:
  
  \begin{equation}
  \delta \mathbf{x}_k = \text{difference}(\mathbf{x}_{k+1}, \mathbf{x}_k) = \mathbf{x}_{k+1} \ominus \mathbf{x}_k
  \end{equation}

where $\mathbf{x} \in M$ (the manifold) and $\delta \mathbf{x} \in T_\mathbf{x} M$ (the tangent space at $\mathbf{x}$).

To compute the necessary derivatives during optimization, we also define the Jacobians of these operators with respect to their arguments:

- The Jacobian of the **integration** operator with respect to the first and second arguments:
  
  \begin{equation}
  \frac{\partial (\mathbf{x} \oplus \delta \mathbf{x})}{\partial \mathbf{x}}, \frac{\partial (\mathbf{x} \oplus \delta \mathbf{x})}{\partial \delta \mathbf{x}} = J_{\text{integrate}}(\mathbf{x}, \delta \mathbf{x})
  \end{equation}

- The Jacobian of the **difference** operator with respect to the first and second arguments:

  \begin{equation}
  \frac{\partial (\mathbf{x}_2 \ominus \mathbf{x}_1)}{\partial \mathbf{x}_1}, \frac{\partial (\mathbf{x}_2 \ominus \mathbf{x}_1)}{\partial \mathbf{x}_1} = J_{\text{difference}}(\mathbf{x}_2, \mathbf{x}_1)
  \end{equation}

  With this understanding, we can now explore **how to implement a state class in Crocoddyl** in `03_optimizing_an_unicycle.ipynb`.

---

### III. Actuation Models in Crocoddyl

Work in progress. More details in `04_actuating_an_acrobot.ipynb`.

---

### IV. Activation and Residual Models in Crocoddyl

Work in progress. More details in `07_targetting_in_aerial_manipulation.ipynb`.

---

### V. Inverse vs Forward Dynamics

Work in progress. More details in `08_backflips_in_humanoids.ipynb`.

## Reference

The material presented in this Notebook was previously presented at the ICRA at 2020. For more information, please read the following paper:

C. Mastalli et al. Crocoddyl: An Efficient and Versatile Framework for Multi-Contact Optimal Control, 2020

In [ ]:
from IPython.display import HTML

# Youtube
HTML(
    '<iframe width="560" height="315" src="https://www.youtube.com/embed/nTeiHioEO4w" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'
)



![title](https://cmastalli.github.io/assets/img/publications/astronaut360_v2.png)